# Lack of Desert Wells? Seems sus...

I've been completely unable to find any desert wells in my game, even using Chunkbase. So I'm just going to [comb my world](https://www.youtube.com/watch?v=UQ7TnQBSV00) for suspicious sand.

## Imports, Setup and Macros

In [1]:
import json
from collections import Counter, defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict, List, Tuple

import matplotlib.pyplot as plt
import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
save_folder = Path(environ["SAVE_PATH"])

In [3]:
def summarize_keystore(keystore: Dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: Dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, Dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

In [4]:
def split_block_value(value: int) -> List[int]:
    """Split a 64-bit int into 16x 4-bit ints

    Parameters
    ----------
    value: 64-bit int
        The int to split

    Returns
    -------
    list of 16x 4-bit ints
    """
    small_ints: List[int] = []
    for _ in range(16):
        value, remainder = divmod(value, 2**4)
        small_ints.append(remainder)
    return small_ints

In [5]:
def get_region_and_chunk(
    coords: tuple[int, int, int]
) -> tuple[tuple[int, int], tuple[int, int, int]]:
    """Given a set of coordinates return the corresponding
    region file and chunk

    Parameters
    ----------
    (int, int, int)
        The block coordinates

    Returns
    -------
    str
        The region index
    (int, int, int)
        The chunk index (including y-sliced sub-chunk)
    """
    chunk: tuple[int, int, int] = tuple(coord // 16 for coord in coords)  # type: ignore
    region = (chunk[0] // 32, chunk[2] // 32)
    return region, chunk

## Chunk Contents

I'm taking a similare approach to my [Iron Vein finding notebook](Finding%20an%20Iron%20Vein.ipynb), combing through each chunklet and finding the ones containing the block I want (suspicious sand).

In [6]:
def search_for_sand(path: Path) -> list[tuple[tuple[int, int, int], nbt.TAG_Compound]]:
    sus_chunklets = []
    try:
        for chunk in region.RegionFile(path).iter_chunks():
            for section in chunk["sections"]:
                if "block_states" not in section:
                    continue
                if "minecraft:suspicious_sand" in [
                    block["Name"].value for block in section["block_states"]["palette"]
                ]:
                    coords = (
                        chunk["xPos"].value * 16,
                        section["Y"].value * 16,
                        chunk["zPos"].value * 16,
                    )
                    sus_chunklets.append((coords, section))
                    # if len(sus_chunklets) <= 10:
                    #     print(f"Found sus sand somewhere around {coords}")
    except UnicodeDecodeError:
        # nbt library can't handle unicode
        print("Skipping", path.name)
    return sus_chunklets

In [7]:
%%time
suspicious_sand_chunklets = []
futures = []
# can't use PPE because chunks can't be pickled
with ThreadPoolExecutor(max_workers=7) as executor:
    for path in sorted((save_folder / "region").glob("*")):
        futures.append(executor.submit(search_for_sand, path))

for result in as_completed(futures):
    suspicious_sand_chunklets.extend(result.result())

print(f"Found {len(suspicious_sand_chunklets)} chunklets with suspicious sand")

Skipping r.-3.-8.mca
Skipping r.-4.-8.mca
Skipping r.0.-5.mca
Found 256 chunklets with suspicious sand
CPU times: user 9min 46s, sys: 32.6 s, total: 10min 19s
Wall time: 9min 32s


Not all of these will be desert wells, of course, but we can do some filtering by biome.

In [8]:
desert_chunklets = []
for coords, chunklet in suspicious_sand_chunklets:
    if "minecraft:desert" in [biome.value for biome in chunklet["biomes"]["palette"]]:
        desert_chunklets.append((coords, chunklet))

print(f"Found {len(desert_chunklets)} chunklets at least partially in the desert")

Found 27 chunklets at least partially in the desert


Welp, nothing left tol do but get out my comb.

In [15]:
with Path("sus_sand.txt").open("w") as f:
    f.write("desert?, X, Y, Z\n")
    for coord, chunklet in suspicious_sand_chunklets:
        is_desert = "minecraft:desert" in [
            biome.value for biome in chunklet["biomes"]["palette"]
        ]
        f.write(("Y" if is_desert else " ") + ", " + str(coord)[1:-1] + "\n")

## Soon...

![Well would you look at that...](../_static/well.png)